In [1]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [2]:
def createC1(dataSet):
    C1=[]
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return map(frozenset,C1)
#Frozensets are sets that are frozen, which means they’re immutable; 
#you can’t change them. You need to use the type frozenset instead of set because you’ll later use these sets as the key in a dictionary; 
#you can do that with frozensets but not with sets

In [3]:
def scanD(D,Ck,minSupport):#D为dataset;Ck为 a list of candidate sets
    ssCnt={}
    for tid in D:#每一个transaction
        for can in Ck:#每一个候选
            if can.issubset(tid):
                if not ssCnt.has_key(can):
                    ssCnt[can]=1
                else:
                    ssCnt[can]+=1
    numItems=float(len(D))
    retList=[]
    supportData={}
    for key in ssCnt:
        support=ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key]=support
    return retList,supportData

In [4]:
def aprioriGen(Lk,k):
    retList=[]
    lenLk=len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1=list(Lk[i])[:k-2]
            L2=list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2:
                retList.append(Lk[i]|Lk[j])
    return retList

In [5]:
def apriori(dataSet,minSupport=0.5):
    C1=createC1(dataSet)
    D=map(set,dataSet)
    L1,supportData=scanD(D,C1,minSupport)
    L=[L1]
    k=2
    while(len(L[k-2])>0):
        Ck=aprioriGen(L[k-2],k)
        Lk,supK=scanD(D,Ck,minSupport)
        supportData.update(supK)#添加supK到supportData
        L.append(Lk)
        k+=1
    return L,supportData

In [6]:
dataSet=loadDataSet()

In [7]:
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [8]:
C1=createC1(dataSet)

In [9]:
D=map(set,dataSet)

In [10]:
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [11]:
L1,suppData0=scanD(D,C1,0.5)

In [12]:
L1

[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]

In [13]:
suppData0

{frozenset({4}): 0.25,
 frozenset({5}): 0.75,
 frozenset({2}): 0.75,
 frozenset({3}): 0.75,
 frozenset({1}): 0.5}

In [14]:
L,suppData=apriori(dataSet)

In [15]:
L

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 [frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({3, 5})],
 [frozenset({2, 3, 5})],
 []]

In [16]:
suppData

{frozenset({5}): 0.75,
 frozenset({3}): 0.75,
 frozenset({2, 3, 5}): 0.5,
 frozenset({1, 2}): 0.25,
 frozenset({1, 5}): 0.25,
 frozenset({3, 5}): 0.5,
 frozenset({4}): 0.25,
 frozenset({2, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({1}): 0.5,
 frozenset({1, 3}): 0.5,
 frozenset({2}): 0.75}

In [24]:
def calcConf(freqSet,H,supportData,brl,minConf=0.7):
    prunedH=[]
    for conseq in H:
        conf=supportData[freqSet]/supportData[freqSet-conseq]
        if conf>=minConf:
            print freqSet-conseq,'-->',conseq,'conf:',conf
            brl.append((freqSet-conseq,conseq,conf))
            prunedH.append(conseq)
    return prunedH

In [20]:
def rulesFromConseq(freqSet,H,supportData,brl,minConf=0.7):
    m=len(H[0])
    if (len(freqSet)>(m+1)):
        Hmp1=aprioriGen(H,m+1)
        Hmp1=calcConf(freqSet,Hmp1,supportData,brl,minConf)
        if len(Hmp1)>1:
            rulesFromConseq(freqSet,Hmp1,supportData,brl,minConf)

In [22]:
def generateRules(L,supportData,minConf=0.7):
    bigRuleList=[]
    for i in range(1,len(L)):
        for freqSet in L[i]:
            H1=[frozenset([item]) for item in freqSet]
            if i>1:#可进一步合并
                rulesFromConseq(freqSet,H1,supportData,bigRuleList,minConf)
            else:#计算confidence
                calcConf(freqSet,H1,supportData,bigRuleList,minConf)
    return bigRuleList

In [26]:
rules=generateRules(L,suppData,minConf=0.7)

frozenset([1]) --> frozenset([3]) conf: 1.0
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0


In [27]:
rules=generateRules(L,suppData,minConf=0.5)

frozenset([3]) --> frozenset([1]) conf: 0.666666666667
frozenset([1]) --> frozenset([3]) conf: 1.0
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0
frozenset([3]) --> frozenset([2]) conf: 0.666666666667
frozenset([2]) --> frozenset([3]) conf: 0.666666666667
frozenset([5]) --> frozenset([3]) conf: 0.666666666667
frozenset([3]) --> frozenset([5]) conf: 0.666666666667
frozenset([5]) --> frozenset([2, 3]) conf: 0.666666666667
frozenset([3]) --> frozenset([2, 5]) conf: 0.666666666667
frozenset([2]) --> frozenset([3, 5]) conf: 0.666666666667


In [31]:
mushDatSet=[line.split() for line in open('mushroom.dat').readlines()]

In [35]:
from numpy import *
matrix(mushDatSet)

matrix([['1', '3', '9', ..., '98', '107', '113'],
        ['2', '3', '9', ..., '99', '108', '114'],
        ['2', '4', '9', ..., '99', '108', '115'],
        ..., 
        ['2', '6', '9', ..., '106', '112', '119'],
        ['1', '7', '10', ..., '102', '110', '119'],
        ['2', '3', '9', ..., '104', '112', '119']], 
       dtype='|S3')

In [36]:
L,suppData=apriori(mushDatSet,minSupport=0.3)

In [39]:
for item in L[2]:
    if item.intersection('2'):
        print item

frozenset(['90', '2', '23'])
frozenset(['39', '2', '53'])
frozenset(['2', '59', '34'])
frozenset(['2', '53', '85'])
frozenset(['23', '2', '34'])
frozenset(['2', '53', '34'])
frozenset(['93', '2', '86'])
frozenset(['39', '2', '59'])
frozenset(['2', '59', '36'])
frozenset(['39', '2', '36'])
frozenset(['2', '28', '53'])
frozenset(['86', '2', '59'])
frozenset(['90', '2', '59'])
frozenset(['2', '93', '63'])
frozenset(['90', '39', '2'])
frozenset(['39', '2', '93'])
frozenset(['76', '2', '34'])
frozenset(['39', '2', '85'])
frozenset(['2', '85', '67'])
frozenset(['63', '90', '2'])
frozenset(['93', '2', '36'])
frozenset(['2', '63', '34'])
frozenset(['2', '86', '23'])
frozenset(['2', '59', '23'])
frozenset(['39', '2', '67'])
frozenset(['63', '39', '2'])
frozenset(['2', '28', '85'])
frozenset(['28', '2', '59'])
frozenset(['2', '63', '23'])
frozenset(['2', '86', '53'])
frozenset(['2', '63', '85'])
frozenset(['2', '36', '34'])
frozenset(['2', '86', '67'])
frozenset(['28', '2', '86'])
frozenset(['93